## Part 1: Preprocessing

In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [26]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [27]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [59]:
display(y_df["Attrition"].value_counts())
display(y_df["Department"].value_counts())

Attrition
No     1233
Yes     237
Name: count, dtype: int64

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [28]:
# Create a list of at least 10 column names to use as X data
x_cols = attrition_df.drop(columns=["Attrition", "Department"]).columns

# Create X_df using your selected columns
X_df = attrition_df[x_cols]

# Show the data types for X_df
X_df.dtypes

Age                          int64
BusinessTravel              object
DistanceFromHome             int64
Education                    int64
EducationField              object
EnvironmentSatisfaction      int64
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole           int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
dtype: object

In [29]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [30]:
# testing/debugging code
display(X_train.shape)
display(X_test.shape)

(1102, 25)

(368, 25)

In [31]:
# Convert your X data to numeric data types however you see fit
# collecting all non-int64 objects into a list
from numpy import int64
from sklearn.preprocessing import OneHotEncoder

# getting all non numeric columns
non_numeric_cols = X_df.dtypes[X_df.dtypes == "object"].index.tolist()

# encoding them
encoder = OneHotEncoder(sparse_output=False)
X_train_enc = pd.DataFrame(encoder.fit_transform(X_train[non_numeric_cols]), 
                           columns=encoder.get_feature_names_out(non_numeric_cols), index=X_train.index)
X_test_enc = pd.DataFrame(encoder.transform(X_test[non_numeric_cols]),
                          columns=encoder.get_feature_names_out(non_numeric_cols), index=X_test.index)

# sticking them back to the original df
X_train_encoded = X_train.merge(X_train_enc, left_index=True, right_index=True)
X_train_encoded.drop(non_numeric_cols, axis=1, inplace=True)

X_test_encoded = X_test.merge(X_test_enc, left_index=True, right_index=True)
X_test_encoded.drop(non_numeric_cols, axis=1, inplace=True)

X_train_encoded.head()
# STOPPED HERE: SHAPES ARE DIFFERENT after merge OR concat operation, I suspect a handful of indexes get mixed around in the operation

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
56,35,18,5,2,71,3,3,1,1,22,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1447,36,15,4,4,88,1,2,4,1,24,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1000,52,8,4,3,54,3,1,1,9,13,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
281,42,1,1,2,99,3,2,3,1,25,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1296,30,10,3,1,64,3,3,3,9,14,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [32]:
# testing/debugging code
display(X_train_enc.shape)
display(X_test_enc.shape)
display(X_train_encoded.shape)
display(X_test_encoded.shape)

(1102, 23)

(368, 23)

(1102, 43)

(368, 43)

In [33]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_encoded_scaled = scaler.fit_transform(X_train_encoded)

# Scale the training and testing data
X_test_encoded_scaled = scaler.transform(X_test_encoded)

In [34]:
display(X_train_encoded_scaled.shape)
display(X_test_encoded_scaled.shape)

(1102, 43)

(368, 43)

In [35]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
ohe = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
y_train_department = ohe.fit_transform(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = pd.DataFrame(y_train_department, columns=ohe.get_feature_names_out(['Department']))

y_train_department.head()


,Department_Research & Development,Department_Sales
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,1.0,0.0


In [36]:
# doing the same for the testing departments
y_test_department = ohe.transform(y_test[['Department']])
y_test_department = pd.DataFrame(y_test_department, columns=ohe.get_feature_names_out(['Department']))
y_test_department.head()


,Department_Research & Development,Department_Sales
0,1.0,0.0
1,1.0,0.0
2,0.0,0.0
3,1.0,0.0
4,1.0,0.0


In [37]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
y_train_attrition = ohe_attrition.fit_transform(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = pd.DataFrame(y_train_attrition, columns=ohe_attrition.get_feature_names_out(["Attrition"]))
y_train_attrition.head()

,Attrition_Yes
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [38]:
# doing the same for the test attritions
y_test_attrition = ohe_attrition.transform(y_test[["Attrition"]])
y_test_attrition = pd.DataFrame(y_test_attrition, columns=ohe_attrition.get_feature_names_out(["Attrition"]))
y_test_attrition.head()

,Attrition_Yes
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


## Part 2: Create, Compile, and Train the Model

In [39]:
# Find the number of columns in the X training data.
input_features = len(X_train_encoded_scaled[0])

# Create the input layer
input_layer = layers.Input(shape=(input_features,), name='input_layer')

# Create at least two shared layers
dense_shared_1 = layers.Dense(units=4, activation='relu')(input_layer)
dense_shared_2 = layers.Dense(units=4, activation='relu')(dense_shared_1)

In [48]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(4, activation='relu')(dense_shared_2)

# Create the output layer
department_output = layers.Dense(len(y_train_department.columns), activation="softmax", name="department_output")(department_dense)

In [49]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(4, activation='relu')(dense_shared_2)

# Create the output layer
attrition_output = layers.Dense(len(y_train_attrition.columns), activation='sigmoid', name="attrition_output")(attrition_dense)

In [50]:
# Create the model
model = Model(inputs=input_layer, outputs={
    "department_output": department_output,
    "attrition_output": attrition_output 
})

# Compile the model
model.compile(optimizer="adam",
            loss={
                "attrition_output": "binary_crossentropy",
                "department_output": "categorical_crossentropy"
            },
            metrics={
                "attrition_output": "accuracy",
                "department_output": "accuracy" 
                })

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 43)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 4)         │        176 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 4)         │         20 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 4)         │         20 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 4)         │         20 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │          5 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 2)         │         10 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 251 (1004.00 B)

 Trainable params: 251 (1004.00 B)

 Non-trainable params: 0 (0.00 B)

In [51]:
# Train the model
model.fit(
    X_train_encoded_scaled,
    {
        "attrition_output": y_train_attrition,
        "department_output": y_train_department 
    },
    epochs=10,  # You can adjust the number of epochs based on your needs
    batch_size=32,  # You can adjust the batch size based on your available memory
    validation_split=0.2  # You can specify the validation split if you have a separate validation set
)

Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - attrition_output_accuracy: 0.7498 - attrition_output_loss: 0.6649 - department_output_accuracy: 0.6649 - department_output_loss: 0.6500 - loss: 1.3149 - val_attrition_output_accuracy: 0.8326 - val_attrition_output_loss: 0.6303 - val_department_output_accuracy: 0.7466 - val_department_output_loss: 0.6371 - val_loss: 1.2673
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8239 - attrition_output_loss: 0.6446 - department_output_accuracy: 0.6784 - department_output_loss: 0.6318 - loss: 1.2765 - val_attrition_output_accuracy: 0.8507 - val_attrition_output_loss: 0.6004 - val_department_output_accuracy: 0.7466 - val_department_output_loss: 0.6171 - val_loss: 1.2175
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8292 - attrition_output_loss: 0.6124 - department_output_accuracy: 0.6930 - department_output_loss: 0.6043 - loss: 1.2168 - val_attrition_output_accuracy: 0.8507 - va

In [55]:
display(X_test_encoded_scaled.shape)
display(y_test_attrition.shape)
display(y_test_department.shape)

(368, 43)

(368, 1)

(368, 2)

In [56]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_encoded_scaled, {
        "attrition_output": y_test_attrition,
        "department_output": y_test_department 
    })

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8799 - attrition_output_loss: 0.3827 - department_output_accuracy: 0.8710 - department_output_loss: 0.3044 - loss: 0.6884 


In [57]:
# Print the accuracy for both department and attrition
print("Accuracy scores for Attrition:", results[0:2])
print("Accuracy for Departments:", results[2:])

Accuracy scores for Attrition: [0.6860388517379761, 0.39089730381965637]
Accuracy for Departments: [0.28654658794403076, 0.8722826242446899, 0.8614130616188049]


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not a good metric because the data is highly imbalanced. There are very little yes labels in the attrition column and very little human resources labels in the department column. A better metric to trained would have been balanced accuracy to account for the imbalanced dataset.
2. For the department output layer I used softmax because its a multiclassification problem, and the model must predict 1 of 3 results. For predicting attrition, I used a sigmoid activation function because it is good for binary classification problems.
3. The model could be improved by giving it more epochs to train on. The more epochs the better the model will perform. Also, the model could have been trained with a better metric. A better metric would improve the low accuracy scores for some of the classes. You could also test differenct configurations of number of layers in the hidden layer and number of nodes to see which configuration improves the NN the most.